## Preprocessing

In [31]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [32]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
first_application_df= application_df.drop(columns = ['EIN', 'NAME', 'USE_CASE', 'SPECIAL_CONSIDERATIONS'])
first_application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,5000,1
1,T3,Independent,C2000,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Trust,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,1,0,5000,0
34295,T4,CompanySponsored,C3000,Association,1,0,5000,0
34296,T3,CompanySponsored,C2000,Association,1,0,5000,0
34297,T5,Independent,C3000,Association,1,0,5000,1


In [33]:
# Determine the number of unique values in each column.
first_application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [34]:
# Look at APPLICATION_TYPE value counts for binning
type_counts=  first_application_df['APPLICATION_TYPE'].value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [35]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_counts.iloc[8:].index)
cutoff_value = 1000
# Replace in dataframe
for app in application_types_to_replace:
    first_application_df['APPLICATION_TYPE'] = first_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
first_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [36]:
# Look at INCOME_AMT value counts for binning
Income_types = first_application_df['INCOME_AMT'].value_counts()
Income_types

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [37]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
incomes_to_replace = list(Income_types.iloc[5:].index)
cutoff_value_classifications = 1883

# Replace in dataframe
for cls in incomes_to_replace:
    first_application_df['INCOME_AMT'] = first_application_df['INCOME_AMT'].replace(cls,"Other")
    
# Check to make sure binning was successful
first_application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             1107
1M-5M              955
1-9999             728
Name: INCOME_AMT, dtype: int64

In [38]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(first_application_df)
dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1,5000,1,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [39]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL'].values
X = dummies.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [40]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                4950      
                                                                 
 dense_4 (Dense)             (None, 80)                4080      
                                                                 
 dense_5 (Dense)             (None, 1)                 81        
                                                                 
Total params: 9111 (35.59 KB)
Trainable params: 9111 (35.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [43]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.5771 - accuracy: 0.7130
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5574 - accuracy: 0.7254
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5545 - accuracy: 0.7276
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5528 - accuracy: 0.7284
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5519 - accuracy: 0.7290
Epoch 6/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5505 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 13s 16ms/step - loss: 0.5493 - accuracy: 0.7300
Epoch 8/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5485 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5480 - accuracy: 0.7307
Epoch 10/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5476 - ac

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5647 - accuracy: 0.7305 - 340ms/epoch - 1ms/step
Loss: 0.5646555423736572, Accuracy: 0.7304956316947937


In [51]:
# Drop the non-beneficial ID columns
cleaned_application_df= application_df.drop(columns = ['EIN', 'NAME', 'ASK_AMT', 'USE_CASE', 'SPECIAL_CONSIDERATIONS'])
cleaned_application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,1
1,T3,Independent,C2000,Co-operative,1,1-9999,1
2,T5,CompanySponsored,C3000,Association,1,0,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,1
4,T3,Independent,C1000,Trust,1,100000-499999,1
...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,1,0,0
34295,T4,CompanySponsored,C3000,Association,1,0,0
34296,T3,CompanySponsored,C2000,Association,1,0,0
34297,T5,Independent,C3000,Association,1,0,1


In [52]:
cleaned_application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
ORGANIZATION         4
STATUS               2
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64

In [53]:
# Look at APPLICATION_TYPE value counts for binning
type_counts=  cleaned_application_df['APPLICATION_TYPE'].value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [54]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace_two = list(type_counts.iloc[8:].index)
cutoff_value = 2000
# Replace in dataframe
for app in application_types_to_replace:
    cleaned_application_df['APPLICATION_TYPE'] = cleaned_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
cleaned_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [55]:
# Look at CLASSIFICATION value counts for binning
classification_types = first_application_df['CLASSIFICATION'].value_counts()
classification_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [56]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_to_replace = list(classification_types.iloc[5:].index)
cutoff_classifications = 25000

# Replace in dataframe
for cls in classification_to_replace:
    cleaned_application_df['CLASSIFICATION'] = cleaned_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
cleaned_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [57]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(cleaned_application_df)
dummies.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [58]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL'].values
X = dummies.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [59]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 500
hidden_nodes_layer2 = 800

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 500)               18000     
                                                                 
 dense_7 (Dense)             (None, 800)               400800    
                                                                 
 dense_8 (Dense)             (None, 1)                 801       
                                                                 
Total params: 419601 (1.60 MB)
Trainable params: 419601 (1.60 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [62]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=180)


Epoch 1/180
804/804 [==============================] - 8s 8ms/step - loss: 0.5743 - accuracy: 0.7196
Epoch 2/180
804/804 [==============================] - 7s 9ms/step - loss: 0.5604 - accuracy: 0.7272
Epoch 3/180
804/804 [==============================] - 5s 6ms/step - loss: 0.5586 - accuracy: 0.7274
Epoch 4/180
804/804 [==============================] - 6s 7ms/step - loss: 0.5574 - accuracy: 0.7303
Epoch 5/180
804/804 [==============================] - 7s 8ms/step - loss: 0.5561 - accuracy: 0.7288
Epoch 6/180
804/804 [==============================] - 6s 8ms/step - loss: 0.5553 - accuracy: 0.7289
Epoch 7/180
804/804 [==============================] - 6s 8ms/step - loss: 0.5556 - accuracy: 0.7298
Epoch 8/180
804/804 [==============================] - 7s 8ms/step - loss: 0.5541 - accuracy: 0.7286
Epoch 9/180
804/804 [==============================] - 8s 10ms/step - loss: 0.5538 - accuracy: 0.7299
Epoch 10/180
804/804 [==============================] - 8s 9ms/step - loss: 0.5536 - accur

In [63]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5960 - accuracy: 0.7368 - 998ms/epoch - 4ms/step
Loss: 0.5960170030593872, Accuracy: 0.7367929816246033


Third Attempt

In [66]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
third_application_df= application_df.drop(columns = ['EIN', 'USE_CASE', 'STATUS'])
third_application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,Association,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Co-operative,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,Association,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Trust,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Trust,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,Association,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,Association,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Association,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,Association,0,N,5000,1


In [67]:
# Determine the number of unique values in each column.
third_application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [68]:
# Look at NAME value counts for binning
type_counts=  third_application_df['NAME'].value_counts()
type_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [69]:
# Choose a cutoff value and create a list of name types to be replaced
# use the variable name `name_types_to_replace`
name_types_to_replace = list(type_counts.iloc[8:].index)
cutoff_value = 7889
# Replace in dataframe
for app in name_types_to_replace:
    third_application_df['NAME'] = third_application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
third_application_df['NAME'].value_counts()

Other                                              29662
PARENT BOOSTER USA INC                              1260
TOPS CLUB INC                                        765
UNITED STATES BOWLING CONGRESS INC                   700
WASHINGTON STATE UNIVERSITY                          492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC      408
PTA TEXAS CONGRESS                                   368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC        331
ALPHA PHI SIGMA                                      313
Name: NAME, dtype: int64

In [70]:
# Look at APPLICATION_TYPE value counts for binning
type_counts=  third_application_df['APPLICATION_TYPE'].value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [71]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
applications_types_to_replace = list(type_counts.iloc[8:].index)
cutoff_value = 8474
# Replace in dataframe
for app in applications_types_to_replace:
    third_application_df['APPLICATION_TYPE'] = third_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
third_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [72]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(third_application_df)
dummies.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_PTA TEXAS CONGRESS,NAME_SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,NAME_TOPS CLUB INC,NAME_UNITED STATES BOWLING CONGRESS INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [73]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL'].values
X = dummies.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [74]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [75]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 909
hidden_nodes_layer2 = 398

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 909)               101808    
                                                                 
 dense_10 (Dense)            (None, 398)               362180    
                                                                 
 dense_11 (Dense)            (None, 1)                 399       
                                                                 
Total params: 464387 (1.77 MB)
Trainable params: 464387 (1.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [76]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [77]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5381 - accuracy: 0.7339
Epoch 2/100
804/804 [==============================] - 12s 14ms/step - loss: 0.5207 - accuracy: 0.7442
Epoch 3/100
804/804 [==============================] - 12s 15ms/step - loss: 0.5180 - accuracy: 0.7458
Epoch 4/100
804/804 [==============================] - 12s 15ms/step - loss: 0.5166 - accuracy: 0.7465
Epoch 5/100
804/804 [==============================] - 12s 16ms/step - loss: 0.5149 - accuracy: 0.7469
Epoch 6/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5141 - accuracy: 0.7481
Epoch 7/100
804/804 [==============================] - 13s 16ms/step - loss: 0.5135 - accuracy: 0.7489
Epoch 8/100
804/804 [==============================] - 13s 16ms/step - loss: 0.5131 - accuracy: 0.7482
Epoch 9/100
804/804 [==============================] - 15s 18ms/step - loss: 0.5118 - accuracy: 0.7486
Epoch 10/100
804/804 [==============================] - 15s 18ms/step - l

In [78]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 0.9633 - accuracy: 0.7366 - 2s/epoch - 7ms/step
Loss: 0.9633270502090454, Accuracy: 0.7365597486495972


In [79]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")


/Applications/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
